In [1]:
"""
Configs:
    # Env
    env_id="NNWorld01-v01", 
     - once in the goal state, each action ends the episode and returns a reward of 5
     - no clear episodes, continuously train the agent with num_train_steps, 
       if the agent reaches terminal, just reset environment and keep training 
     - try to immitate the bandit experiments
    # Params
    eps_sched_fn=poly(0.5), lr_sched_fn=poly(0.8)
    # Algos
    haver2, action_sigma=adaptive(1), haver_delta=0.01, haver_const=varied
Status:
"""

from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=5, suppress=True)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
sns.set_palette("tab20")
colors = sns.color_palette("bright")

import time
from tqdm import tqdm 
import multiprocessing

# import gymnasium as gym
import gym
import gym_examples
from gym.wrappers import FlattenObservation

from algos import *
from bandit_problem import *
from utils import *

In [ ]:

random.seed(123)
np.random.seed(123)
tdqm_disable = True

# params
num_trials = 1
num_steps_train = 5000
num_episodes_eval = 100

lr_sched_type = "linear"
lr_sched_fn = create_lr_sched_fn(lr_sched_type, lr=0.7)

max_eps = 1.0
min_eps = 1.0
decay_rate = 0.0001
eps_sched_type = "poly"
eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)

# create gym env
env_id = "gym_examples/NNWorldEnv01-v1"
env_scheme = "two_island"
gamma = 0.95

num_depths = 4
num_widths = 16
num_actions = num_widths
terminal_reward = -10.0

reward_dist = "normal"
problem_instance = "multi_gap_nonlinear"
action_max_mu = 0.0
action_sigma = 1.0
action_sigmas = action_sigma*np.ones(num_actions)
gap_splits = [0.5]
gap_deltas = [5.0]
bandit_problem = BanditProblem(
    problem_instance, reward_dist, num_actions, action_max_mu, 
    action_sigmas=action_sigmas, gap_splits=gap_splits, gap_deltas=gap_deltas)
print(f"action_mus = {bandit_problem.action_mus}")
print(f"action_sigmas = {bandit_problem.action_sigmas}")

action_max_mu = bandit_problem.action_mus[0]
optimal_num_steps = num_depths
optimal_vstar = terminal_reward*gamma**(optimal_num_steps-1) \
    + action_max_mu*np.sum([gamma**k for k in range(optimal_num_steps-1)])
optimal_reward_per_step = (terminal_reward + action_max_mu*(optimal_num_steps-1))/optimal_num_steps  
print(f"optimal_num_steps = {optimal_num_steps}")
print(f"optimal_reward_per_step = {optimal_reward_per_step}")
print(f"optimal_vstar = {optimal_vstar}")

env = gym.make(env_id, num_depths=num_depths, num_widths=num_widths, 
               bandit_problem=bandit_problem, terminal_reward=terminal_reward)
env_wrapped = FlattenObservation(env)
cur_state, info = env_wrapped.reset()

manager = multiprocessing.Manager()
episode_start_sigmahats_list = manager.list()
episode_rewards_list = manager.list()
episode_vstar_est_list = manager.list()
Q_table_list = manager.list()
Q_nvisits_list = manager.list()    

def run_trial(i_trial, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    # env = gym.make(env_id, size=gridworld_size)
    # env_wrapped = FlattenObservation(env)
    # env_wrapped.reset(seed=10000+i_trial)

    # lr_sched_fn = create_lr_sched_fn(lr_sched_type)
    # eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)
    q_algo = create_q_algo(args["est_name"])

    Q_table, Q_nvisits, stats = q_algo(
        env_wrapped, num_actions, num_steps_train,
        gamma, lr_sched_fn, eps_sched_fn, tdqm_disable, args)

    episode_start_sigmahats, episode_rewards, episode_vstar_est= zip(*stats)
    episode_start_sigmahats_list.append(episode_start_sigmahats)
    episode_rewards_list.append(episode_rewards)
    episode_vstar_est_list.append(episode_vstar_est)
    Q_table_list.append(Q_table)
    Q_nvisits_list.append(Q_nvisits)

args = dict()
args["action_sigma"] = action_sigma
args["haver_alpha"] = 2.0
args["haver_delta"] = 0.05
args["haver_const"] = 1.0
args["weightedms_num_data"] = 1000
args["num_depths"] = num_depths
args["env_scheme"] = env_scheme

pool = multiprocessing.Pool()

episode_start_sigmahats_dict = defaultdict()
episode_rewards_dict = defaultdict()
episode_vstar_est_dict = defaultdict()
episode_vstar_est_bias_dict = defaultdict()
episode_vstar_est_var_dict = defaultdict()
episode_vstar_est_mse_dict = defaultdict()
Q_table_dict = defaultdict()
Q_nvisits_dict = defaultdict()

haver_const_ary = [1.0]
haver_name_ary = [f"haver_{x}" for x in haver_const_ary]
haver3_name_ary = [f"haver3_{x}" for x in haver_const_ary]

est_name_ary = ["max", "weightedms"]
# est_name_ary = haver_name_ary + est_name_ary 
# est_name_ary = est_name_ary + haver_name_ary
est_name_ary = est_name_ary + haver3_name_ary
est_name_ary = ["haver3_1.0", "weightedms"]
for est_name in est_name_ary:
    start_time = time.time()
    print(f"\n-> est_name = {est_name}")
    if "haver" in est_name:
        elems = est_name.split("_")
        args["est_name"] = elems[0]
        args["haver_const"] = float(elems[-1])
        print(f"haver_const = {args['haver_const']}")
    else:
        args["est_name"] = est_name
    
    pool.starmap(run_trial, [(i, args) for i in range(num_trials)])

    episode_start_sigmahats_ary = np.hstack([episode_start_sigmahats_list])
    episode_rewards_ary = np.hstack([episode_rewards_list])
    episode_vstar_est_ary = np.hstack([episode_vstar_est_list])

    episode_start_sigmahats_dict[est_name] = np.mean(episode_start_sigmahats_ary, 0)
    episode_rewards_dict[est_name] = np.mean(episode_rewards_ary, 0)
    episode_vstar_est_dict[est_name] = np.mean(episode_vstar_est_ary, 0)
    print(f"last_episode_start_sigmahat = {episode_start_sigmahats_dict[est_name][-1]:.4f}")
    print(f"last_episode_reward_per_step = {episode_rewards_dict[est_name][-1]:.4f}")
    print(f"last_episode_estim_start_muhat = {episode_vstar_est_dict[est_name][-1]:.4f}")
    
    episode_vstar_est_bias_dict[est_name] = np.mean(episode_vstar_est_ary - optimal_vstar, 0)
    episode_vstar_est_var_dict[est_name] = np.var(episode_vstar_est_ary - optimal_vstar, 0, ddof=1)
    episode_vstar_est_mse_dict[est_name] = \
        episode_vstar_est_bias_dict[est_name]**2 \
        + episode_vstar_est_var_dict[est_name]
    
    # Q_table_dict[est_name] = np.mean(np.stack(Q_table_list), 0)
    # Q_nvisits_dict[est_name] = np.mean(np.stack(Q_nvisits_list), 0)
    # print(np.stack(Q_table_list).shape)
    Q_table_dict[est_name] = np.stack(Q_table_list)[0,:,:,:]
    Q_nvisits_dict[est_name] = np.stack(Q_nvisits_list)[0,:,:,:]
    # print(Q_table_list[0][0,0,:])
    # print(Q_table_list[1][0,0,:])
    # print(Q_table_ary)
    # stop
                           
    episode_start_sigmahats_list[:] = []
    episode_rewards_list[:] = []
    episode_vstar_est_list[:] = []
    Q_table_list[:] = []
    Q_nvisits_list[:] = []
    
    end_time = time.time()
    print(f"it takes {end_time-start_time:0.4f}")

action_mus = [ 0.  0.  0.  0.  0.  0.  0.  0. -5. -5. -5. -5. -5. -5. -5. -5.]
action_sigmas = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
optimal_num_steps = 4
optimal_reward_per_step = -2.5
optimal_vstar = -8.573749999999999

-> i_step = 0
cur_state = [0 0]
action = 5
reward = 0.24
new_state = [1 5]
haver3_estimator
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[-inf -inf -inf -inf -inf -inf -inf -inf]
[0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001]
Q_nvisits[cur_state][action] = 1.0
Q_table[cur_state][action], before = 0.00
Q_table[cur_state][action], after = 0.24
haver3_estimator
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ -inf  -inf  -inf  -inf  -inf 0.237  -inf  -inf  -inf  -inf  -inf  -inf
  -inf  -inf  -inf  -inf]
[0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001
 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001 0.00001]
haver3_estimator
action_maxlcb_idx = 5
action_max

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



action = 1
reward = 0.92
new_state = [1 1]
haver3_estimator
[1. 1. 1. 3. 1. 1. 2. 1.]
[1. 1. 1. 3. 1. 1. 2. 1.]
[-0.36052  0.73431 -0.74553 -0.1237  -0.86917  1.1666  -8.10221 -0.87352]
[0.00001 0.00001 0.00001 0.30029 0.00001 0.00001 0.21649 0.00001]
haver3_estimator
action_maxlcb_idx = 5
action_maxlcb_muhat = 1.17
Bset_idxes = [5]
Bset_probs = [0. 0. 0. 0. 0. 1. 0. 0.]
Bset_nvisits = [0. 0. 0. 0. 0. 1. 0. 0.]
Q_est = 1.17
weightedms_estimator
idxes = [5]
probs = [0. 0. 0. 0. 0. 1. 0. 0.]
Q_est = 1.17
Q_nvisits[cur_state][action] = 12.0
Q_table[cur_state][action], before = 0.72
Q_table[cur_state][action], after = 0.83
haver3_estimator
[ 2. 12.  2.  1.  1.  1.  2.  1.  2.  1.  1.  1.  1.  1.  1.  1.]
[ 2. 12.  2.  1.  1.  1.  2.  1.  2.  1.  1.  1.  1.  1.  1.  1.]
[ 0.04131  0.82536 -0.73162 -3.06116 -0.06171  0.237   -0.13406 -1.40093
 -8.57193 -5.51234 -4.68091 -3.60479 -4.43462 -4.18285 -5.12814 -3.83091]
[0.98449 0.82754 0.22313 0.00001 0.00001 0.00001 1.25489 0.00001 2.53379
 0.0

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Bset_probs = [0.06818 0.77273 0.      0.      0.04545 0.02273 0.04545 0.      0.
 0.      0.      0.      0.04545 0.      0.      0.     ]
Bset_nvisits = [ 3. 34.  0.  0.  2.  1.  2.  0.  0.  0.  0.  0.  2.  0.  0.  0.]
Q_est = -0.40
weightedms_estimator
idxes = [ 0  1  4  5  6 12]
probs = [0.166 0.361 0.    0.    0.02  0.158 0.239 0.    0.    0.    0.    0.
 0.056 0.    0.    0.   ]
Q_est = -0.63

-> i_step = 225
cur_state = [1 1]
action = 5
reward = 0.39
new_state = [2 5]
haver3_estimator
[2. 1. 2. 2. 2. 1. 2. 2.]
[2. 1. 2. 2. 2. 1. 2. 2.]
[ -9.45756 -10.67657  -5.91607  -9.09205  -9.58108  -8.24433  -5.4274
  -4.36854]
[0.51659 0.00001 5.56289 0.12657 0.40075 0.00001 4.75565 4.48865]
haver3_estimator
action_maxlcb_idx = 5
action_maxlcb_muhat = -8.24
Bset_idxes = [2, 5, 6, 7]
Bset_probs = [0.      0.      0.28571 0.      0.      0.14286 0.28571 0.28571]
Bset_nvisits = [0. 0. 2. 0. 0. 1. 2. 2.]
Q_est = -5.67
weightedms_estimator
idxes = [2 5 6 7]
probs = [0.    0.    0.31  0.    0.   

...

haver3_estimator
[ 2.  4. 25.  1. 41.  3.  4.  7.]
[ 2.  4. 25.  1. 41.  3.  4.  7.]
[ -9.4946   -9.48805  -9.27989  -9.77392  -9.27327 -10.31124  -9.85651
  -9.15394]
[0.32541 1.20779 1.10749 0.00001 1.81061 0.60657 1.0903  0.74445]
haver3_estimator
action_maxlcb_idx = 3
action_maxlcb_muhat = -9.77
Bset_idxes = [0, 1, 2, 3, 4, 5, 6, 7]
Bset_probs = [0.02299 0.04598 0.28736 0.01149 0.47126 0.03448 0.04598 0.08046]
Bset_nvisits = [ 2.  4. 25.  1. 41.  3.  4.  7.]
Q_est = -9.35
weightedms_estimator
idxes = [0 1 2 4 5 6 7]
probs = [0.031 0.175 0.223 0.    0.299 0.004 0.083 0.185]
Q_est = -9.35
Q_nvisits[cur_state][action] = 45.0
Q_table[cur_state][action], before = -7.68
Q_table[cur_state][action], after = -7.73
haver3_estimator
[ 55. 259.  42.   5.  56. 127.  54.  34.   3.   2.   4.   4.   4.   3.
   2.   3.]
[ 55. 259.  42.   5.  56. 127.  54.  34.   3.   2.   4.   4.   4.   3.
   2.   3.]
[ -5.13083  -5.10941  -5.13291  -7.03459  -5.1442   -5.11313  -5.1045
  -5.17668  -9.55206  -8.056

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



action = 6
reward = -0.39
new_state = [1 6]
haver3_estimator
[ 3.  4.  8.  4.  3. 29.  4.  1.]
[ 3.  4.  8.  4.  3. 29.  4.  1.]
[-8.44868 -8.88437 -7.95518 -7.97753 -7.9255  -7.94987 -9.03995 -8.42614]
[0.5264  1.52006 2.86186 1.22995 1.13913 2.04362 0.35063 0.00001]
haver3_estimator
action_maxlcb_idx = 7
action_maxlcb_muhat = -8.43
Bset_idxes = [0, 1, 2, 3, 4, 5, 7]
Bset_probs = [0.05769 0.07692 0.15385 0.07692 0.05769 0.55769 0.      0.01923]
Bset_nvisits = [ 3.  4.  8.  4.  3. 29.  0.  1.]
Q_est = -8.06
weightedms_estimator
idxes = [0 1 2 3 4 5 7]
probs = [0.021 0.078 0.327 0.14  0.169 0.261 0.    0.004]
Q_est = -8.04
Q_nvisits[cur_state][action] = 57.0
Q_table[cur_state][action], before = -5.20
Q_table[cur_state][action], after = -5.25
haver3_estimator
[ 57. 272.  44.   5.  58. 131.  57.  35.   3.   2.   4.   5.   4.   3.
   2.   3.]
[ 57. 272.  44.   5.  58. 131.  57.  35.   3.   2.   4.   5.   4.   3.
   2.   3.]
[ -5.23066  -5.19841  -5.21334  -7.03459  -5.22719  -5.19989  -5.2

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



haver3_estimator
action_maxlcb_idx = 1
action_maxlcb_muhat = -5.29
Bset_idxes = [0, 1, 2, 3, 4, 5, 6, 7, 9, 12, 13, 14, 15]
Bset_probs = [0.08559 0.40228 0.06562 0.00713 0.08845 0.19401 0.08417 0.05136 0.
 0.00285 0.      0.      0.00571 0.00428 0.00428 0.00428]
Bset_nvisits = [ 60. 282.  46.   5.  62. 136.  59.  36.   0.   2.   0.   0.   4.   3.
   3.   3.]
Q_est = -5.40
weightedms_estimator
idxes = [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15]
probs = [0.099 0.084 0.132 0.013 0.125 0.121 0.138 0.144 0.    0.005 0.004 0.005
 0.081 0.013 0.015 0.021]
Q_est = -5.99

-> i_step = 2850
cur_state = [2 2]
action = 6
reward = 0.79
new_state = [3 6]
Q_nvisits[cur_state][action] = 3.0
Q_table[cur_state][action], before = -9.57
Q_table[cur_state][action], after = -9.29
haver3_estimator
[ 60. 282.  46.   5.  62. 136.  59.  36.   3.   2.   4.   5.   4.   3.
   3.   3.]
[ 60. 282.  46.   5.  62. 136.  59.  36.   3.   2.   4.   5.   4.   3.
   3.   3.]
[ -5.33933  -5.29323  -5.30152  -7.03459  -5.3

In [ ]:

fig, axes = fig, axes = plt.subplots(
        nrows=5, ncols=1, sharex=True, sharey=False, figsize=(8,12))
# axes = [axes]

x_ary = np.linspace(4000, num_steps_train-1, num=100, dtype=np.int32)
# est_name_ary = ["weightedms"]
est_name_ary_unwanted = []
est_name_ary_wanted = [est_name for est_name in est_name_ary[:8] \
                 if est_name not in est_name_ary_unwanted]
for est_name in est_name_ary_wanted:
    # axes[0].plot(x_ary, episode_rewards_dict[est_name][x_ary], label=est_name)
    y_ary = running_avg(episode_rewards_dict[est_name][x_ary], 100)
    axes[0].plot(
        x_ary, y_ary, label=est_name)
    axes[1].plot(x_ary, episode_vstar_est_dict[est_name][x_ary], label=est_name)
    axes[2].plot(x_ary, episode_vstar_est_mse_dict[est_name][x_ary], label=est_name)
    axes[3].plot(x_ary, episode_vstar_est_bias_dict[est_name][x_ary], label=est_name)
    axes[4].plot(x_ary, episode_vstar_est_var_dict[est_name][x_ary], label=est_name)

axes[0].axhline(y=optimal_reward_per_step, color="black")
axes[1].axhline(y=optimal_vstar, color="black")
axes[0].set_title(f"reward_per_step, K={num_actions}, num_depths={num_depths}, sigma={action_sigma},"
                    f" delta={gap_deltas[0]}")
axes[1].set_title("vstar_est")
axes[2].set_title("vstar_est_mse")
axes[3].set_title("vstar_est_bias")
axes[4].set_title("vstar_est_var")
# axes[0].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[1].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# axes[2].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:

fig, axes = fig, axes = plt.subplots(
        nrows=5, ncols=1, sharex=True, sharey=False, figsize=(8,12))
# axes = [axes]

x_ary = np.linspace(4000, num_steps_train-1, num=100, dtype=np.int32)
# est_name_ary = ["weightedms"]
est_name_ary_unwanted = ["weightedms", "weightedms2"]
est_name_ary_wanted = [est_name for est_name in est_name_ary[:8] \
                 if est_name not in est_name_ary_unwanted]
for est_name in est_name_ary_wanted:
    # axes[0].plot(x_ary, episode_rewards_dict[est_name][x_ary], label=est_name)
    y_ary = running_avg(episode_rewards_dict[est_name][x_ary], 100)
    axes[0].plot(
        x_ary, y_ary, label=est_name)
    axes[1].plot(x_ary, episode_vstar_est_dict[est_name][x_ary], label=est_name)
    axes[2].plot(x_ary, episode_vstar_est_mse_dict[est_name][x_ary], label=est_name)
    axes[3].plot(x_ary, episode_vstar_est_bias_dict[est_name][x_ary], label=est_name)
    axes[4].plot(x_ary, episode_vstar_est_var_dict[est_name][x_ary], label=est_name)

axes[0].axhline(y=optimal_reward_per_step, color="black")
axes[1].axhline(y=optimal_vstar, color="black")
axes[0].set_title(f"reward_per_step, K={num_actions}, num_depths={num_depths}, sigma={action_sigma},"
                    f" delta={gap_deltas[0]}")
axes[1].set_title("vstar_est")
axes[2].set_title("vstar_est_mse")
axes[3].set_title("vstar_est_bias")
axes[4].set_title("vstar_est_var")
# axes[0].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[1].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# axes[2].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:

for est_name in est_name_ary:
    print(f"\n-> est_name = {est_name}")
    Q_table = Q_table_dict[est_name]
    Q_nvisits = Q_nvisits_dict[est_name]
    for i_row in range(num_depths):
        for j_col in range(num_actions):
            print(f"{i_row} {j_col}: {Q_table[i_row,j_col]}, \n     {Q_nvisits[i_row,j_col]}")

 